In [3]:
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import torch 
%matplotlib inline
import torchvision

import torch.nn as nn

In [4]:
training = np.load('reg_multi.npy')

In [5]:
x_dataset = torch.from_numpy(training[:,:16])#.t()
y_dataset = torch.from_numpy(training[:,16])#.t()#.view(-1,1)
number_columns = 16
print(training.shape)
print(x_dataset.shape)
print(y_dataset.shape)
print(x_dataset.size(0) == y_dataset.size(0))

(60000, 17)
torch.Size([60000, 16])
torch.Size([60000])
True


In [25]:
batch_size = 15000
dataset = torch.utils.data.TensorDataset(x_dataset, y_dataset)
train_loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=True)
print(dataset[0:1])

(tensor([[-0.6842, -0.8948,  0.0370, -0.7918, -0.4853, -0.8350, -0.1086,  0.7443,
          0.1368,  0.7213,  0.6274, -0.2482,  0.9813,  0.1931, -0.1693, -0.4865]]), tensor([-0.5732]))


In [26]:
# f = Ax + b
A = torch.randn((1,number_columns),requires_grad=True)
b = torch.randn(1,requires_grad=True)

In [27]:
loss = torch.nn.MSELoss()

# model = torch.nn.Sequential(torch.nn.Linear(number_columns,512),
#                     torch.nn.ReLU(), torch.nn.Linear(512,1))
input_size = number_columns
hidden_sizes = [512, 128]
output_size = 1
# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size))
                      # nn.Softmax(dim=1))
c_loss = 0.0

In [31]:
optimizer = torch.optim.SGD(model.parameters(),lr =0.1)

max_epoch = 100
for epoch in range(max_epoch):
    for datum in train_loader:
        optimizer.zero_grad()
        (features,target) = datum
        y_pred = model(features)
        c_loss = loss(y_pred, target)
        c_loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"t = {epoch}, loss = {c_loss}")

t = 0, loss = 0.30824968218803406
t = 10, loss = 0.31136736273765564
t = 20, loss = 0.308852881193161
t = 30, loss = 0.3123956322669983
t = 40, loss = 0.2962976396083832
t = 50, loss = 0.3159068822860718
t = 60, loss = 0.3072340488433838
t = 70, loss = 0.3057303726673126
t = 80, loss = 0.3112246096134186
t = 90, loss = 0.30244180560112


In [29]:
print(f"loss = {c_loss}")
# print(f"A = {A.detach().numpy()}, b = {b.item()}")

loss = 0.3121108114719391


In [30]:
torch.save(model, "multi_regression.pt")